In [134]:
%load_ext autoreload

%autoreload 2

from bottom_up.all import *

dev = [json.loads(_) for _ in open("dev.jsonl")]

#slens = get_slens(dev, bottom_up_from_corpus_nops, dep_probs=dep_probs)

#nn = EasyAllenNLP()

out = get_features_and_labels("mini.train.jsonl", cutoff=5000)

v = DictVectorizer(sparse=False)

X = v.fit_transform([_["feats"] for _ in out])

y = np.asarray([_["y"] for _ in out])

clf = LogisticRegression(random_state=0,
                         solver='lbfgs',
                         C=1,
                         multi_class='ovr').fit(X, y)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
f1_experiment(dev, bottom_up_from_clf, clf=clf, v=v)

[*] Index error
[*] Index error
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning heuristic 
[*] Index error
warning he

0.8382707437779342

In [71]:
lr = f1_experiment_error_analysis(dev, bottom_up_from_clf, clf=clf, v=v)

[*] Index error
[*] Index error
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
warning heuristic
[*] Index error
wa

In [9]:
from code.printers import pretty_print_conl
lr.sort(key=lambda x:x[0])
ix = 12
f1, s, predicted = lr[ix]
pretty_print_conl(s)
print(f1, s["q"], predicted, s["compression_indexes"], s["r"], s["headline"], ",", " ".join([(_["word"] + "_" + _["pos"] + "_{}".format(str(_["index"]))) for _ in s["tokens"]]))

1	February	<-nsubj-	confident
2	07	<-nummod-	February
3	,	<-punct-	February
4	2011	<-nummod-	February
5	-	<-punct-	February
6	With	<-case-	Maaf
7	7	<-nummod-	Maaf
8	Khoon	<-compound-	Maaf
9	Maaf	<-nmod-	February
10	ready	<-amod-	Maaf
11	to	<-mark-	hit
12	hit	<-xcomp-	ready
13	the	<-det-	theatres
14	theatres	<-dobj-	hit
15	next	<-amod-	week
16	week	<-nmod:tmod-	hit
17	,	<-punct-	Maaf
18	Priyanka	<-compound-	Chopra
19	Chopra	<-appos-	Maaf
20	is	<-cop-	confident
21	all	<-dep-	confident
22	confident	<-ROOT-	ROOT
23	about	<-case-	performance
24	her	<-nmod:poss-	performance
25	performance	<-nmod-	confident
26	and	<-cc-	confident
27	thinks	<-conj-	confident
28	she	<-nsubj-	best
29	's	<-cop-	best
30	the	<-det-	best
31	best	<-ccomp-	thinks
32	in	<-case-	Bollywood
33	Bollywood	<-nmod-	best
34	.	<-punct-	confident
0.3333333333333333 [18, 19] {1, 2, 6, 7, 9, 18, 19, 20, 22} [18, 19, 20, 26, 27, 28, 29, 30, 31] 45 Priyanka Chopra thinks she's the best , February_NNP_1 07_CD_2 ,_,_3 2011_CD_4 -_:_5 

In [94]:
all_ = []

for itm in lr:
    f1, s, predicted = itm
    original_ixs = [_["index"] for _ in s["tokens"]]
    y_true = [_ in s['compression_indexes'] for _ in original_ixs]
    y_pred = [_ in predicted for _ in original_ixs]

    for tok in s["tokens"]:
        gold = tok["index"]  in s["compression_indexes"]
        pred = tok["index"] in predicted
        y = gold == pred
        all_.append((y, tok, s, pred, gold))

In [95]:
pose = set([_[1]["pos"] for _ in all_])

tmp_ = []

for i in pose:
    bypos = np.mean([_[0] for _ in all_ if _[1]["pos"] == i])
    tot = len([_[0] for _ in all_ if _[1]["pos"] == i])
    tmp_.append((i, bypos, tot))
    
tmp_.sort(key=lambda x:x[1])

print(tmp_)

[('SYM', 0.0, 1), ('PDT', 0.6666666666666666, 6), ('$', 0.717391304347826, 46), ('RBS', 0.75, 8), ('LS', 0.7777777777777778, 18), ('WDT', 0.7821782178217822, 101), ('WP', 0.8125, 64), ('WRB', 0.8181818181818182, 44), ('CC', 0.8265107212475633, 513), ('JJ', 0.8267937219730942, 1784), ('CD', 0.8322884012539185, 638), ('NN', 0.8331399396611743, 4309), ('EX', 0.8333333333333334, 6), ('RB', 0.8394004282655246, 467), ('PRP', 0.8480243161094225, 329), ('NNS', 0.8497156783103168, 1231), ('IN', 0.8607634543178974, 3196), ('VBP', 0.8652849740932642, 193), ('VBG', 0.8711256117455138, 613), ('JJS', 0.875, 48), ('VBD', 0.8808104886769964, 839), ('DT', 0.8843164151806127, 2187), ('PRP$', 0.8861788617886179, 369), ('JJR', 0.890625, 64), ('VBZ', 0.8964028776978418, 695), ('TO', 0.8975903614457831, 664), ('POS', 0.9045454545454545, 220), ('NNPS', 0.905511811023622, 127), ('NNP', 0.9057239057239057, 5049), ('VBN', 0.9066496163682864, 782), ('VB', 0.9113372093023255, 688), ('RP', 0.927536231884058, 69), 

In [96]:
im = [_ for _ in all_ if _[1]["pos"] == i and _[0] == False]

from collections import Counter

for _ in im:
    y, tok, s, pred, gold = _
    prev_ = tok["index"] - 2
    next_ = tok["index"] + 2
    toks = s["tokens"][prev_:next_]
    if tok["word"] != "and":
        print(" ".join([_["word"] for _ in toks]), "pred=", pred, "gold=",gold, tok["word"])

, but he 's pred= True gold= False but
, but Leland residents pred= True gold= False but
, but private employers pred= True gold= False but
settle or exchange two pred= False gold= True or
, but the purpose pred= True gold= False but
Reds vs Future Reds pred= False gold= True vs
- or so said pred= True gold= False or
Love & Lust Sleepover pred= False gold= True &
. And it 's pred= True gold= False And


In [66]:
import string
string.punctuation
[_ for _ in string.punctuation]

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']